# Критерии остановки

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

Можно останавливать эксперимент при достижении 95% уверенности, что целевая метрики одной группы лучше других.  
Отметка 95% произвольна. Можно останавливать при 90%. Можно рассмотреть более объективный критерий остановки.

Можно перевести вероятности в ожидаемый эффект.  
Но выбор порога остается.   
Более удобный вопрос - есть ли смысл продолжать дальше?

Качественные соображения:  
Каждый день эксперимента - потери за счет попадания в худшую группу.  
Так же потери времени и усложнение поддержки/разработки (сложно оценить).  

Каждый день эксперимента - новые данные и уточнение оценок метрик.  
Ценность данных с течением времени неодинакова.  
Добавление первых 100 точек дает много информации, добавление 100 точек к 10 млн. ничего не меняет в оценке.  

Решение об остановке должно учитывать последствия выбора.    
При тестировании лекарств нужна большая уверенность в отсутствии побочек. Высока стоимость ошибки.  
В веб-сервисах выбранных вариант продержится, например, год до повторного или следующего эксперимента.  
  

В итоге цена поддержания эксперимента постоянна, ценность новых данных падает.  
Когда ценность новых данных падает ниже стоимости поддержания эксперимента и выгоды/потерь после прекращения,
эксперимент пора останавливать.

Конкретная реализация зависит от рассматриваемых метрик.  
Пример для конверсий.  
Правдоподобие удобно задавать биномиальным распределением, априорное распределение - бета-распределением.  
Тогда апостериорное также будет бета-распределением. 

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
P(\mathcal{H} | \mathcal{D}) = P(p | n_s, N) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
$$

$$
\mbox{Beta}(x; \alpha, \beta) \equiv \frac{x^{\alpha-1} (1 - x)^{\beta-1}}{\int_0^1 dx x^{\alpha-1} (1 - x)^{\beta-1}}
 = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1 - x)^{\beta-1}
$$

Разность приближенно можно считать нормальным распределением.

$$
\begin{split}
P(p_B > p_A) & = P(p_B - p_A > 0)
\\
\\
P_{p_A}(x) = \mbox{Beta}(x; n_{s_A} + \alpha, N_A - n_{s_A} + \beta)
& \approx \mbox{Norm}(x; \mu_A, \sigma_A^2),
\quad
\mu_A = (n_{s_A} + \alpha - 1) /N_A, 
\,
\sigma_A^2 = \mu_A (1 - \mu_A) / N_A,
\quad
N_A \gg n_{s_A} \gg 1
\\
\\
P_{p_B}(x) = \mbox{Beta}(x; n_{s_B} + \alpha, N_B - n_{s_B} + \beta)
& \approx \mbox{Norm}(x; \mu_B, \sigma_B^2),
\quad
\mu_B = (n_{s_B} + \alpha - 1)/N_B, 
\,
\sigma_B^2 = \mu_B (1 - \mu_B) / N_B,
\quad
N_B \gg n_{s_B} \gg 1
\\
\\
P_{p_B - p_A}(x) = 
\int_{-\infty}^{\infty} dy P_{p_B}(y) P_{p_A}(y-x)
& \approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right),
\quad
\mbox{Norm}(x ; \mu, \sigma^2) \equiv \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
\\
\\
P(p_B - p_A > 0) & = 1 - F_{p_B - p_A}(0)
\end{split}
$$

По мере набора данных апостериорные распределения сужаются.  
То же для разности.  
На графике полные распределения при N=1000 и 10000.  
Также средние и 95% области наибольшей плотности вероятности по мере набора N.  
На графиках полных распределений графики при N=1000 шире N=10000.  
По мере набора данных 95% области наибольшей плотности вероятности сужается.

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df[['pa_lower', 'pa_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sa'], r['N'])), axis=1, result_type="expand")
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df[['pb_lower', 'pb_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sb'], r['N'])), axis=1, result_type="expand")
df['pb_gt_pa'] = df.apply(lambda r: prob_pb_gt_pa(posterior_dist_binom(r['sa'], r['N']), posterior_dist_binom(r['sb'], r['N']), post_samp=10_000), axis=1)
df['diff_mu'] = df.apply(lambda r: r['pb'] - r['pa'], axis=1)
df['diff_s'] = df.apply(lambda r: np.sqrt(posterior_dist_binom(r['sa'], r['N']).std()**2 + posterior_dist_binom(r['sb'], r['N']).std()**2), axis=1)
df[['diff_lower', 'diff_upper']] = df.apply(lambda r: (r['diff_mu'] - 2*r['diff_s'], r['diff_mu'] + 2*r['diff_s']), axis=1, result_type="expand")
#todo: loss

xaxis_min = -0.05
xaxis_max = 0.15  
#x = np.linspace(0, xaxis_max, 1000)
x = np.linspace(xaxis_min, xaxis_max, 1000)
fig = go.Figure()
for N, o in [(1000, 0.3), (10000, 1)]:
    dist_a = posterior_dist_binom(df[df['N'] == N]['sa'], N)
    dist_b = posterior_dist_binom(df[df['N'] == N]['sb'], N)
    fig.add_trace(go.Scatter(x=x, y=dist_a.pdf(x), line_color='red', opacity=o, name=f'А, N={N}'))
    fig.add_trace(go.Scatter(x=x, y=dist_b.pdf(x), line_color='blue', opacity=o, name=f'Б, N={N}'))
    #
    diff = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])
    fig.add_trace(go.Scatter(x=x, y=diff.pdf(x), line_color='black', opacity=o, name=f'Diff, N={N}'))
    fig.update_layout(title='Апостериорные распределения',
                      xaxis_title='$p$',
                      yaxis_title='Плотность вероятности',
                      xaxis_range=[xaxis_min, xaxis_max],
                      hovermode="x",
                      height=500)
#     fig.update_layout(title='Апостериорные распределения',
#                       xaxis_title='$p$',
#                       yaxis_title='Плотность вероятности',
#                       xaxis_range=[0, xaxis_max],
#                       hovermode="x",
#                       height=500)
fig.show()


# xaxis_min = -0.05
# xaxis_max = 0.15    
# x = np.linspace(xaxis_min, xaxis_max, 1000)
# fig = go.Figure()
# for N in [1000, 10000]:
#     diff = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])
#     fig.add_trace(go.Scatter(x=x, y=diff.pdf(x), line_color='black', name=f'Diff, N={N}'))
#     fig.update_layout(title='Апостериорные распределения',
#                       xaxis_title='$p$',
#                       yaxis_title='Плотность вероятности',
#                       xaxis_range=[xaxis_min, xaxis_max],
#                       hovermode="x",
#                       height=500)
# fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pa'], name='A',
                         line_color='red'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pa_upper']) + list(reversed(df['pa_lower'])),
                         fill="toself", name='A, 95% PDI', marker_color='red', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['pb'], name='B',
                         line_color='blue'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pb_upper']) + list(reversed(df['pb_lower'])),
                         fill="toself", name='B, 95% PDI', marker_color='blue', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_mu'], name='Diff',
                         line_color='black'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['diff_lower']) + list(reversed(df['diff_upper'])),
                         fill="toself", name='A, 95% PDI', marker_color='black', opacity=0.2))
fig.update_layout(title='$p_A, p_B$',
                  yaxis_tickformat = ',.1%',
                  xaxis_title='N',
                  height=700)
fig.show()


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['N'], y=df['diff_mu'], name='$p_B - p_A$',
#                          line_color='black'))
# fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
#                          y=list(df['diff_lower']) + list(reversed(df['diff_upper'])),
#                          fill="toself", name='$p_B - p_A, \mbox{ 95% PDI}$', marker_color='black', opacity=0.2))
# fig.update_layout(title='$p_B - p_A$',
#                   yaxis_tickformat = ',.1%',
#                   xaxis_title='N')
# fig.show()


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'], name='P(pb > pa)',
#                          line_color='black'))
# fig.update_layout(title='$P(p_B > p_A)$',
#                   yaxis_range=[0, 1],
#                   xaxis_title='N')
# fig.show()

Дисперсия бета-распределения $Beta(x; \alpha, \beta)$:

$$
Var(Beta(x; \alpha, \beta)) = \frac{\alpha \beta}{(\alpha + \beta)^2(\alpha + \beta + 1)}
$$

При $N > n_s \gg \alpha_0, \beta_0$:

$$
\alpha \approx n_s, \qquad \beta \approx N - n_s, \qquad n_s \approx p N
\\
Var(Beta(x; \alpha, \beta)) = \frac{n_s (N - n_s)}{N^3} = \frac{p (1 - p)}{N}
$$

Приближенно $Var(Beta) \sim 1/N$.  

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_s'],
                         line_color='black', name='sigma'))
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_s'][0] * np.sqrt(df['N'][0] / df['N']), 
                         line_color='black', mode='lines', line_dash='dash', name='sigma0 * sqrt(N0/N)'))
fig.update_layout(title='Стандартное отклонение pB - pA',
                  xaxis_title='$N$',
                  #yaxis_title='sigma',
                  #xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)  
fig.show()

По мере набора данных:  
-Средние в выборках стремятся к точному среднему  
-Дисперсии сужаются ~1/N  
-Разность средних стремится к точной разности  
-Дисперсия разности сужается ~1/N  
-Растет P(pb>pa)  
-Уменьшаются ожидаемые потери.  
-Стоимость данных остается постоянной.  

Пусть на текущий момент есть N точек и оценки апостериорных распределений.  
-Можно считать, что pb-pa не изменится. На самом деле оно будет меняться, но нельзя сказать как. 
Е[pb-pa] = pb-pa не изменится, можно использовать его.  
-Дисперсия будет убывать ~1/N.  

Ожидаемая разность не меняется, поэтому оценка эффекта после остановки тоже не меняется.  
Уменьшение дисперсии дает снижение возможных потерь при том же среднем эффекте.  
На графике вероятность pb>pa - площадь правее нуля, ожидаемые потери - среднее в части меньше нуля.

Можно посчитать изменение P(pb>pa) и ожидаемых потерь.  

In [ ]:
N = 5000
approx_diff_dist = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])


# x = np.linspace(-0.3, 0.3, 1000)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
#                          line_color='black', name='$\mbox{Аналитическое приближение}$'))
# # fig.add_trace(go.Scatter(x=[approx_diff_dist.mean()[0], approx_diff_dist.mean()[0]], 
# #                          y=[0, max(approx_diff_dist.pdf(x))], 
# #                          line_color='black', mode='lines', name='mean'))
# fig.add_trace(go.Scatter(x=x[x<0], y=approx_diff_dist.pdf(x[x<0]), fill='tozeroy',
#                          line_color='black', opacity=0.3, name='loss'))
# # fig.add_trace(go.Scatter(x=x[x>=0], y=approx_diff_dist.pdf(x[x>=0]), 
# #                          line_color='black', fillcolor='white', name='gain', fill='tozeroy'))
# fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
#                          line_color='black', mode='lines', line_dash='dash', showlegend=False))
# fig.update_layout(title='$p_B - p_A$',
#                   xaxis_title='$x$',
#                   yaxis_title='Плотность вероятности',
#                   xaxis_range=[-0.1, 0.1],
#                   hovermode="x",
#                   height=500)
# fig.show()



x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
mu = pb - pa
N = 5000
s = np.sqrt((pb*(1-pb)/N) + (pa*(1-pa)/N))
d = stats.norm(loc=mu, scale=s)
fig.add_trace(go.Scatter(x=x, y=d.pdf(x), 
                         line_color='black', name=f'N={N}'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(d.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.add_trace(go.Scatter(x=x[x<0], y=d.pdf(x[x<0]), fill='tozeroy',
                         line_color='black', opacity=0.3, name='loss', showlegend=False))
N = 1000
s = np.sqrt((pb*(1-pb)/N) + (pa*(1-pa)/N))
d = stats.norm(loc=mu, scale=s)
fig.add_trace(go.Scatter(x=x, y=d.pdf(x), 
                         line_color='black', opacity=0.3, name=f'N={N}'))
fig.add_trace(go.Scatter(x=x[x<0], y=d.pdf(x[x<0]), fill='tozeroy',
                         line_color='black', opacity=0.3, name='loss'))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.05, 0.05],
                  hovermode="x",
                  height=500)
fig.show()
#plot for fixed mu with N=1000, N=5000 


mu = pb - pa
s = np.sqrt((pb*(1-pb)/df['N']) + (pa*(1-pa)/df['N']))
pbgtpa = [1-stats.norm(loc=mu, scale=s).cdf(0) for s in s]

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_s']*10,
                         line_color='red', name='s*10, simulated'))
fig.add_trace(go.Scatter(x=df['N'], y=s*10,
                         line_color='black', line_dash='dash', name='s*10, fixed mean'))
fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'],
                         line_color='red', name='pb_gt_pa, simulated'))
fig.add_trace(go.Scatter(x=df['N'], y=pbgtpa,
                         line_color='black', line_dash='dash', name='pb_gt_pa, fixed mean'))
#todo: loss
fig.update_layout(title='Стандартное отклонение pB - pA',
                  xaxis_title='$N$',
                  #yaxis_title='sigma',
                  #xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)  
fig.show()

При одинаковом среднем меньше ожидаемых потерь лучше.  

Останавливать, если:  
Ожидаемые потери на след. шаге - Ожидаемые потери сейчас < Стоимости данных  

Стоимость данных - ожидаемые потери из-за попадания части пользователей в худшую группу.  

Потери на след. шаге и стоимость данных можно оценить исходя из того, что дисперсия будет уменьшаться 1/N при том же среднем.

Ожидаемые потери должны учитывать количество пользователей, на которых будут раскатываться изменения, и LTV. 

$$
E[L(p_B - p_A | N + \Delta N)] - E[L(p_B - p_A | N)] < Cost(\Delta N)
\\
\begin{split}
E[L(p_B - p_A | N)] & = LTV * M \int_{-\infty}^0 x P_{p_B - p_A}(x) dx
\\
& \approx LTV * M \int_{-\infty}^0 x Norm(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2 | N) dx
\\
& = LTV * M * I(N)
\end{split}
\\
Cost(\Delta N) \approx LTV * w_A \Delta N * E[P_{p_B - p_A}]  = LTV * w_A (\mu_B - \mu_A) \Delta N
$$

$$
LTV * M (I(N+\Delta N) - I(N)) < LTV w_A (\mu_B - \mu_A) \Delta N
\\
\frac{I(N+\Delta N) - I(N)}{\Delta N} < \frac{w_A}{M} (\mu_B - \mu_A)
$$

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))

a, b = -np.inf, (0 - approx_diff_dist.mean()) / approx_diff_dist.std()
tr_loss = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='Diff'))
fig.add_trace(go.Scatter(x=x, y=tr_loss.pdf(x), 
                         line_color='red', name='Loss'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()


dN = 100

res = []
for N in range(na+nb, 100_000, dN):
    mu = approx_diff_dist.mean()
    s = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)
    new_diff_dist = stats.norm(loc=approx_diff_dist.mean(), scale=s)
    p = 1 - new_diff_dist.cdf(0)
    a, b = -np.inf, (0 - new_diff_dist.mean()) / new_diff_dist.std()
    tr_loss = stats.truncnorm(a=a, b=b, loc=new_diff_dist.mean(), scale=new_diff_dist.std())
    mean_loss = tr_loss.mean() * new_diff_dist.cdf(0)
    s2 = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N+dN)
    new_diff_dist2 = stats.norm(loc=approx_diff_dist.mean(), scale=s2)
    a2, b2 = -np.inf, (0 - new_diff_dist2.mean()) / new_diff_dist2.std()
    tr_loss2 = stats.truncnorm(a=a2, b=b2, loc=new_diff_dist2.mean(), scale=new_diff_dist2.std())
    mean_loss2 = tr_loss2.mean() * new_diff_dist2.cdf(0)
    d_loss = (mean_loss2 - mean_loss) / dN
    loss_diff = new_diff_dist.pdf(0) * s**2 / (2 * N)
    #loss_diff = stats.norm.pdf(mu/s) * s / (2 * N)
    res.append((N, mu, s, p, mean_loss, d_loss, loss_diff))
    
#res = [(N, approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)) for N in range(na+nb, 1000000, 100)]

df_res = pd.DataFrame(res, columns=['N', 'mu', 'std', 'p', 'mean_loss', 'd_loss', 'loss_diff'])
df_res


fig = go.Figure()
#fig.add_trace(go.Scatter(x=df['N'], y=df['std'] * 100, 
#                         line_color='blue', name='std * 100'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['p'], 
                         line_color='black', name='p'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['mean_loss'] * 100, 
                         line_color='red', name='mean_loss * 100'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['mu']*10, 
                         line_color='yellow', name='mu*10'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['d_loss'] * 1000000, 
                         line_color='green', name='d_loss * 1000000'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['loss_diff'] * 1000000, 
                         line_color='purple', name='loss_diff * 1000000'))
fig.show()

Ожидаемые потери аналитически 
$$
\frac{d I(N)}{dN} < \frac{w_A}{M} (\mu_B - \mu_A)
$$

$$
\begin{split}
I(N) &= \int_{-\infty}^{0} x \mathcal N(x; m, \sigma^2) dx
  = \int_{-\infty}^{0} \big(x + m - m\big) \mathcal N(x;m,\sigma^2) dx 
  \\
  & = m \int_{-\infty}^{0} \mathcal N(x;m,\sigma^2) dx + \int_{-\infty}^{0} (x-m) \mathcal N(x;m,\sigma^2) dx 
  \\
  &= m \Phi \left(\tfrac{0-m}{\sigma}\right) + \sigma \int_{-\infty}^{(0-m)/\sigma} t \varphi(t) dt 
  = m \Phi \left(-\tfrac{m}{\sigma}\right) - \sigma \varphi \left(-\tfrac{m}{\sigma}\right)
  \\
  & = m \Phi \left(-\tfrac{m}{\sigma}\right) - \sigma \varphi \left(\tfrac{m}{\sigma}\right)
\end{split}
$$

При $\sigma(N) = \sigma_0 \sqrt{N_0 / N}$:

$$
\begin{split}
I(N) &= m\,\Phi\!\left(-\tfrac{m}{\sigma(N)}\right) \;-\; \sigma(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right), \\[6pt]
\frac{dI}{dN}
&= m\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)\cdot\frac{m\,\sigma'(N)}{\sigma(N)^2}
\;-\;\Big[\sigma'(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
+ \sigma(N)\,\varphi'\!\left(\tfrac{m}{\sigma(N)}\right)\cdot\frac{dz}{dN}\Big], \\[6pt]
&= \frac{m^2\sigma'(N)}{\sigma(N)^2}\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
- \sigma'(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
- \frac{m^2\sigma'(N)}{\sigma(N)^2}\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right), \\[6pt]
&= -\,\sigma'(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right).
\end{split}
\\
\sigma'(N) = -\sigma(N) / 2 N 
\\
\frac{dI}{dN} \;=\; \frac{\sigma(N)}{2N}\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
= \frac{\sigma(N)^2}{2N} Norm\left(0; \mu_B - \mu_A, \sigma^2 \right)
$$

Условие остановки
$$
\frac{\sigma(N)^2}{2N} Norm\left(0; \mu_B - \mu_A, \sigma^2 \right) < \frac{w_A}{M} (\mu_B - \mu_A)
\\
Norm\left(0; \mu_B - \mu_A, \sigma^2 \right) < 2 w_A \frac{N}{M} \frac{(\mu_B - \mu_A)}{\sigma(N)^2}
\\
\phi(z) < 2 w_A \frac{N}{M} z, \quad z = \frac{\mu_B - \mu_A}{\sigma}, \quad \phi(z) = \frac{1}{\sqrt{2\pi}} e^{-x^2/2}
$$

Асимптотика хвостов нормального распределения https://en.wikipedia.org/wiki/Mills_ratio .  
Т.к. $u/z \ge 1, u \in [z, \infty)$

$$
\Phi(-z) 
= \frac{1}{\sqrt{2\pi}}\int_{z}^{\infty} e^{-u^{2}/2}\,du
\;\le\; \frac{1}{\sqrt{2\pi}}\int_{z}^{\infty} \frac{u}{z} e^{-u^{2}/2}\,du
= \frac{1}{z\sqrt{2\pi}} e^{-z^{2}/2}
= \frac{\varphi(z)}{z}, \qquad z>0 .
$$

Условие остановки: вероятность группа B хуже (левый хвост) ограничена сверху $Norm(x)/x$ меньше $2w_A N/M$. 

$$
F(0; \mu_B - \mu_A, \sigma^2) < \frac{\phi(z)}{z} < 2w_A \frac{N}{M}
$$

Считать время на изменение решения.  
Готовы ли держать еще столько времени?  
Это способ оценить расходы на поддержание эксперимента на глаз.

Оценки длительности

Проверки

Ссылки

https://en.wikipedia.org/wiki/Expected_value_of_sample_information  

https://en.wikipedia.org/wiki/Truncated_normal_distribution  

https://en.wikipedia.org/wiki/Mills_ratio

https://en.wikipedia.org/wiki/Conjugate_prior

https://en.wikipedia.org/wiki/Posterior_predictive_distribution

https://en.wikipedia.org/wiki/Beta-binomial_distribution